In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#counts runs and wickets in the last 5 overs
def count_runs_wickets(data):
  runs = data["runs"]
  wickets = data["wicket"]
  l = len(runs)
  if l < 30:
    l = l
  else:
    l = l + 1
  last = 30
  for i in range(0, l):
    if i-30 < 0:
      data.loc[i,"last_runs5"] = runs.iloc[i]
      data.loc[i,"last_wicket5"] = wickets.iloc[i]
    else:
      sr = runs.iloc[i-30:i].sum()
      sw = wickets.iloc[i-30:i].sum()
      data.loc[i-1,"last_runs5"] = sr
      data.loc[i-1,"last_wicket5"] = sw
  return data


In [ ]:
import os

path = '/content/datasets'
path1 = '/content/df/'
files = []
names = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            names.append(file)
            files.append(os.path.join(r, file))



for i in range(0,len(files)):
  with open(files[i],'r') as f:
    with open(path1+names[i],'w') as f1:
        f.readline() # skip header line
        for line in f:
            if line[0] != "i":
              f1.write(line)



In [ ]:
df = pd.DataFrame(columns=['id','overs', 'teamBat', 'striker', 'non-striker', 'baller','wickets', 'teamBowl', 'venue', 'city','runs','cruns','run-rate','total'])
for i in range(0,len(files)):
  print(files[i], names[i])
  df1 = pd.read_csv(files[i],nrows =11)
  df2 = pd.read_csv(path1+names[i], error_bad_lines=False,header=None)
  #df2 = df2.reset_index()
  df2.rename(columns={0:'balls',1:'innings',2:'overs',3:'teamBat',4:'striker',5:'non-striker',6:'baller',7:'curr_runs',8:'extras',9:'wicket',10:'outname'},inplace=True)
  df2 = df2[df2['innings']==1]
  df2.drop(columns=['innings','balls','outname'],inplace=True)
  df1 = df1.reset_index()
  batting = df2.iloc[0]['teamBat']
  if df1.loc[0]['1.3.0'] == batting:
    df1.loc[0]['version'] = "teamBat"
    df1.loc[1]['version'] = "teamBowl"
  else:
    df1.loc[1]['version'] = "teamBat"
    df1.loc[0]['version'] = "teamBowl"

  df1.set_index('version', inplace=True)
  df1.drop(columns = ['index'],inplace=True)
  df1.index.name="" 
  df1 = df1.T
  #df2[['teamBowl','venue','city','winner_runs']] = np.array(df1.iloc[0,:])
  df2['teamBowl'] = str(df1['teamBowl']).split('    ')[1].split("\n")[0]
  df2['venue'] = str(df1['venue']).split('    ')[1].split("\n")[0]
  df2['city'] = str(df1['city']).split('    ')[1].split("\n")[0] 
  df2['runs'] = df2[['curr_runs','extras']].sum(axis=1)
  df2['cruns'] = df2['runs'].cumsum(axis=0)
  df2['run-rate'] = df2['cruns']/df2['overs']
  df2['total'] = df2['curr_runs'].sum() + df2['extras'].sum()
  df2.drop(columns = ['curr_runs','extras'],inplace=True)
  #df2['wickets'].replace(np.nan, 0, inplace=True)
  #df2['wickets'].replace(np.nan, 0, inplace=True)
  df2.loc[~df2['wicket'].isnull(),'wicket'] = 1  # not nan
  df2.loc[df2['wicket'].isnull(),'wicket'] = 0
  df2['wickets'] = df2['wicket'].cumsum(axis=0)
  df2['id'] = i
  df2 = count_runs_wickets(df2)
  df2.drop(columns = ['wicket'],inplace=True)
  df = df.append(df2)


df = df.reset_index()
df.drop(columns=['index'], inplace=True)
df

/content/datasets/520595.csv 520595.csv
/content/datasets/1074968.csv 1074968.csv
/content/datasets/258463.csv 258463.csv
/content/datasets/1213058.csv 1213058.csv
/content/datasets/951367.csv 951367.csv
/content/datasets/1215160.csv 1215160.csv
/content/datasets/895819.csv 895819.csv
/content/datasets/951337.csv 951337.csv
/content/datasets/1198490.csv 1198490.csv
/content/datasets/592276.csv 592276.csv
/content/datasets/1158073.csv 1158073.csv
/content/datasets/1197406.csv 1197406.csv
/content/datasets/1186493.csv 1186493.csv
/content/datasets/1172509.csv 1172509.csv
/content/datasets/951305.csv 951305.csv
/content/datasets/1217742.csv 1217742.csv
/content/datasets/1192876.csv 1192876.csv
/content/datasets/875459.csv 875459.csv
/content/datasets/951335.csv 951335.csv
/content/datasets/1187667.csv 1187667.csv
/content/datasets/1178996.csv 1178996.csv
/content/datasets/657633.csv 657633.csv
/content/datasets/1190768.csv 1190768.csv
/content/datasets/1187013.csv 1187013.csv
/content/dat

,id,overs,teamBat,striker,non-striker,baller,wickets,teamBowl,venue,city,runs,cruns,run-rate,total,last_runs5,last_wicket5
0,0,0.1,Zimbabwe,BRM Taylor,H Masakadza,KD Mills,0,New Zealand,Eden Park,Auckland,1,1,10.000000,159,1.0,0.0
1,0,0.2,Zimbabwe,H Masakadza,BRM Taylor,KD Mills,0,New Zealand,Eden Park,Auckland,0,1,5.000000,159,0.0,0.0
2,0,0.3,Zimbabwe,H Masakadza,BRM Taylor,KD Mills,0,New Zealand,Eden Park,Auckland,1,2,6.666667,159,1.0,0.0
3,0,0.4,Zimbabwe,BRM Taylor,H Masakadza,KD Mills,0,New Zealand,Eden Park,Auckland,0,2,5.000000,159,0.0,0.0
4,0,0.5,Zimbabwe,BRM Taylor,H Masakadza,KD Mills,0,New Zealand,Eden Park,Auckland,1,3,6.000000,159,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101592,838,19.2,Zimbabwe,E Chigumbura,R Mutumbami,Wahab Riaz,6,Pakistan,Gaddafi Stadium,Lahore,0,168,8.750000,172,54.0,2.0
101593,838,19.3,Zimbabwe,AG Cremer,R Mutumbami,Wahab Riaz,6,Pakistan,Gaddafi Stadium,Lahore,1,169,8.756477,172,51.0,2.0
101594,838,19.4,Zimbabwe,R Mutumbami,AG Cremer,Wahab Riaz,6,Pakistan,Gaddafi Stadium,Lahore,1,170,8.762887,172,51.0,2.0
101595,838,19.5,Zimbabwe,AG Cremer,R Mutumbami,Wahab Riaz,6,Pakistan,Gaddafi Stadium,Lahore,1,171,8.769231,172,51.0,2.0


In [ ]:
df
df.to_csv("updated.csv",index=False)